In [25]:
!python3 main.py play --agents q_agent random_agent rule_based_agent rule_based_agent --train 1 --scenario classic --n-rounds 10 --turn-based

size of dictionary: 70
  0%|                                                    | 0/10 [00:00<?, ?it/s]
random
events ['MOVED_DOWN', 'CRATE_REACHED']
###

my_pos  (15, 1)
neigh  [array([14,  1]), array([15,  2]), array([16,  1]), array([15,  0])]
n_values 1  [0, 0, -1, -1]
features [0, 0, 0, 1, -1, -1]

my_pos  (15, 2)
neigh  [array([14,  2]), array([15,  3]), array([16,  2]), array([15,  1])]
n_values 1  [-1, 1, -1, 0]
area [[15, 1], [15, 3], [15, 4], [15, 5]]
count 2
area [[15, 1], [15, 3], [15, 4], [15, 5]]
count 2
features [0, 1, -1, 1, -1, 0]
###
###
old q 0.0
new q 0.1
###

random
events ['WAITED', 'CRATE_WITHOUT_DROPPING_BOMB']
###

my_pos  (15, 2)
neigh  [array([14,  2]), array([15,  3]), array([16,  2]), array([15,  1])]
n_values 1  [-1, 1, -1, 0]
area [[15, 1], [15, 3], [15, 4], [15, 5]]
count 2
area [[15, 1], [15, 3], [15, 4], [15, 5]]
count 2
features [0, 1, -1, 1, -1, 0]

my_pos  (15, 2)
neigh  [array([14,  2]), array([15,  3]), array([16,  2]), array([15,  1])]
n_values 1


random
events ['INVALID_ACTION']
###

my_pos  (14, 1)
neigh  [array([13,  1]), array([14,  2]), array([15,  1]), array([14,  0])]
n_values 1  [0, -1, 0, -1]
mask neighb (array([0, 2]),) [[13  1]
 [15  1]]
bomb_area [[13  1]
 [12  1]
 [11  1]
 [15  1]]
escape i j [1 3] 0
max dist from bomb True
mask neighb (array([0, 2]),) [[13  1]
 [15  1]]
bomb_area [[13  1]
 [12  1]
 [11  1]
 [15  1]]
escape i j [1 3] 2
max dist from bomb False
features [2, 0, 1, -1, 2, -1]

my_pos  (14, 1)
neigh  [array([13,  1]), array([14,  2]), array([15,  1]), array([14,  0])]
n_values 1  [0, -1, 0, -1]
mask neighb (array([0, 2]),) [[13  1]
 [15  1]]
bomb_area [[13  1]
 [12  1]
 [11  1]
 [15  1]]
escape i j [1 3] 0
max dist from bomb True
mask neighb (array([0, 2]),) [[13  1]
 [15  1]]
bomb_area [[13  1]
 [12  1]
 [11  1]
 [15  1]]
escape i j [1 3] 2
max dist from bomb False
features [2, 0, 1, -1, 2, -1]
###
###
old q 0.0
new q -10.0
###

random
collectable coins [[7.e-323]
 [5.e-324]]
events ['INVALID_ACTION']


random
events ['INVALID_ACTION', 'CRATE_WITHOUT_DROPPING_BOMB']
###

my_pos  (3, 1)
neigh  [array([2, 1]), array([3, 2]), array([4, 1]), array([3, 0])]
n_values 1  [0, 0, 1, -1]
area [[2, 1], [1, 1], [4, 1], [5, 1], [6, 1], [3, 2], [3, 3], [3, 4]]
count 3
area [[2, 1], [1, 1], [4, 1], [5, 1], [6, 1], [3, 2], [3, 3], [3, 4]]
count 3
features [0, 1, 0, 0, 1, -1]

my_pos  (3, 1)
neigh  [array([2, 1]), array([3, 2]), array([4, 1]), array([3, 0])]
n_values 1  [0, 0, 1, -1]
area [[2, 1], [1, 1], [4, 1], [5, 1], [6, 1], [3, 2], [3, 3], [3, 4]]
count 3
area [[2, 1], [1, 1], [4, 1], [5, 1], [6, 1], [3, 2], [3, 3], [3, 4]]
count 3
features [0, 1, 0, 0, 1, -1]
###
###
old q 0.0
new q -10.5
###

random
events ['INVALID_ACTION', 'CRATE_WITHOUT_DROPPING_BOMB']
###

my_pos  (3, 1)
neigh  [array([2, 1]), array([3, 2]), array([4, 1]), array([3, 0])]
n_values 1  [0, 0, 1, -1]
area [[2, 1], [1, 1], [4, 1], [5, 1], [6, 1], [3, 2], [3, 3], [3, 4]]
count 3
area [[2, 1], [1, 1], [4, 1], [5, 1], [6, 1], [3, 


random
events ['INVALID_ACTION']
###

my_pos  (2, 1)
neigh  [array([1, 1]), array([2, 2]), array([3, 1]), array([2, 0])]
n_values 1  [0, -1, 1, -1]
mask neighb (array([0]),) [[1 1]]
bomb_area [[1 1]
 [3 1]
 [4 1]
 [5 1]]
escape i j [1 3] 0
max dist from bomb True
features [2, 0, 1, -1, -1, -1]

my_pos  (2, 1)
neigh  [array([1, 1]), array([2, 2]), array([3, 1]), array([2, 0])]
n_values 1  [0, -1, 1, -1]
mask neighb (array([0]),) [[1 1]]
bomb_area [[1 1]
 [3 1]
 [4 1]
 [5 1]]
escape i j [1 3] 0
max dist from bomb True
features [2, 0, 1, -1, -1, -1]
###
###
old q -10.0
new q -19.0
###

random
events ['INVALID_ACTION', 'OPPONENT_ELIMINATED']
###

my_pos  (2, 1)
neigh  [array([1, 1]), array([2, 2]), array([3, 1]), array([2, 0])]
n_values 1  [0, -1, 1, -1]
mask neighb (array([0]),) [[1 1]]
bomb_area [[1 1]
 [3 1]
 [4 1]
 [5 1]]
escape i j [1 3] 0
max dist from bomb True
features [2, 0, 1, -1, -1, -1]

my_pos  (2, 1)
neigh  [array([1, 1]), array([2, 2]), array([3, 1]), array([2, 0])]
n_value

 60%|██████████████████████████▍                 | 6/10 [07:42<05:08, 77.16s/it]


In [26]:
!python3 main.py play --agents q_agent q_agent random_agent rule_based_agent --scenario coin-heaven --n-rounds 5

size of dictionary: 70
nonzero rows: 67
size of dictionary: 70
nonzero rows: 67
  0%|                                                     | 0/5 [00:00<?, ?it/s]/home/vivi/uni/bomberman/agent_code/q_agent/callbacks.py:184: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if (neighbours[j] == game_state['bombs'][0]).all():
100%|█████████████████████████████████████████████| 5/5 [00:47<00:00,  9.42s/it]


In [38]:
#--my-agents --save-replay --turn-based --save-stats

In [ ]:
!python main.py replay "replays/Round 07 (2022-03-23 12-52-13).pt" --turn-based

In [ ]:
reward = np.load("agent_code/q_agent/rewards.npy")
plt.plot(reward,'.')
plt.xlabel("rounds")
plt.ylabel("reward")
#plt.ylim(-100,100)

In [ ]:
model_trained = np.load("agent_code/q_agent/my-saved-model.npy")

In [17]:
import itertools
import numpy as np

v = [0,0, 1, -1,  1, -1]

NEIGHBOURING_FIELDS = [-1,0,1,2] # the possible values for neighbouring fields
GAME_MODE = [0,1,2,3] # the possible values for game mode
CURRENT_FIELD = [0,1,2,3] # the possible values for current filed
dic = {}

def get_arrangements(array):
    return [ np.roll(array, -i) for i in range(0,4) ]

# get all possible combinations
def get_variations(array, r=4):
    return [ item for item in list(itertools.product(array, repeat=r)) ]

# compute state to index for the Q-table
def build_state_to_index(arr1 = NEIGHBOURING_FIELDS, arr2 = GAME_MODE, arr3 = CURRENT_FIELD):
    i = 0
    perm = get_variations(arr1)
    comb = np.array(np.meshgrid(arr2, arr3)).T.reshape(-1,2) # combinations of arr2 and arr3
    step = len(comb)
    for p in perm:
        already_there = []
        arrangements = get_arrangements(p)
        for a in arrangements:
            if tuple(a) in dic.keys():
                already_there.append(True)
            else:
                already_there.append(False)
        if sum(already_there) == 0: # p not in dictionary
            value = [v for v in range(i, i+step)] # value indices for p
            i += step # update i
            dic.update({tuple(p) : value}) # add in dictionary

# give corresponding index and rotation from keys
def get_state_index(state):
    arrangements = get_arrangements(state[-4:]) # rotations of neighbouring fields
    mode_and_field = state[:2]
    value_index = 0
    comb = np.array(np.meshgrid(GAME_MODE, CURRENT_FIELD)).T.reshape(-1,2)
    for i, c in enumerate(comb):
        if (c == mode_and_field).all():
            value_index = i
            break
    for j, a in enumerate(arrangements):
        if tuple(a) in dic.keys():
            print(tuple(a), value_index)
            return dic[tuple(a)][value_index], j
    assert True == 0, state
            
build_state_to_index()
print(get_arrangements(v))
print(get_state_index(v))

[array([ 0,  0,  1, -1,  1, -1]), array([ 0,  1, -1,  1, -1,  0]), array([ 1, -1,  1, -1,  0,  0]), array([-1,  1, -1,  0,  0,  1])]
(-1, 1, -1, 1) 0
(400, 1)


In [4]:
import numpy as np
import random


In [10]:
def dist(pos, objects):
    return np.sqrt( np.power(np.subtract(objects, pos).transpose()[0], 2) + np.power(np.subtract(objects, pos).transpose()[1], 2) )

p = (1,1)
k = [[0,2],[1,2], [1,3], [2,1]]
print(dist(p,k))

print(dist((5,5), (8,5)))

[1.41421356 1.         2.         1.        ]
3.0


In [37]:
def get_neighbours(pos):
    sub = [(0,-1), (-1,0), (-2,0), (0,1)] # left, down, right, up

    neighbours = []
    for i in sub:
        neighbour = np.subtract(pos, i)
        if (0 <= neighbour[0] < 17) and (0 <= neighbour[1] < 17): # check if in game borders
            neighbours.append(neighbour)
    return neighbours, pos

print(get_neighbours((1,1)))

([array([1, 2]), array([2, 1]), array([3, 1]), array([1, 0])], (1, 1))


In [27]:
pos = np.array((1,5))
area = [[1, 3], [1, 4], [1, 5], [1, 2]]
i = -1
for a in area:
    print(a, pos)
    i += 1
    if ((a == pos).all()):
        print(a, pos)
        break
print(i)

[1, 3] [1 5]
[1, 4] [1 5]
[1, 5] [1 5]
[1, 5] [1 5]
2


In [13]:
pos = (1,2)
pos1 = (1,2)
a = [1,2]
b = ((1,3), (1,2), (2,2))
c = any([(pos==b_) for b_ in b])

print(c)
print(pos!=b[2])
print(pos!=pos1)

True
True
False


In [98]:
mask = [[0]]
n = [[0,1],[1,2],[2,1],[1,0]]


True


In [22]:
def foo(i):
    if i > 0:
        if i > 2:
            if i > 4:
                if i > 6:
                    return False
                #return True
            #return True
        return True
    return False       

print(foo(1))

True


In [150]:
a = np.array([1, 3, 1, 7])
c = np.array([[0,1],[1,0],[0,0],[1,1]])
b = np.where(a!=1)
print(c)
d = [[0,1], [1,0], [0,0]]

k = np.where([d_ not in c for d_ in d])[0]
print(np.empty(k))

[[0 1]
 [1 0]
 [0 0]
 [1 1]]
1.8483715e-316


In [68]:
from datetime import datetime

now = datetime.now() # current date and time

year = now.strftime("%Y")
print("year:", year)

month = now.strftime("%m")
print("month:", month)

day = now.strftime("%d")
print("day:", day)

time = now.strftime("%H:%M:%S")
print("time:", time)

date_time = now.strftime("%H_%M_%S")
print("date and time:",date_time)

year: 2022
month: 03
day: 23
time: 18:59:01
date and time: 18_59_01


In [ ]:
import time
start = time.time()

time.sleep(5)
end  = time.time()
print(end-start)